In [15]:
import sys
import os
import math

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from scipy.optimize import curve_fit

from tqdm import tqdm

import concatenate
import plot
import regression as reg
import bootstrap as boot

In [16]:
def readfile(path):
    output = np.loadtxt(f'{path}/analysis/dati.dat', skiprows=1)
    columns = [output[:, i] for i in range(output.shape[1])]

    return np.column_stack(columns)

def format(y, d_y, digits=2):
    d_digits = int(math.floor(math.log10(abs(d_y))))
    d_y_rounded = round(d_y, -d_digits + (digits - 1))

    decimal_places = -d_digits + (digits - 1)

    value_rounded = round(y, decimal_places)

    formatted = f"{value_rounded:.{decimal_places}f}({int(d_y_rounded * 10**decimal_places):0{digits}d})"
    print(formatted)

def boot_fit(x, y, d_y, b_y, model, lim_inf, lim_sup, extension=None):
    x_fit, y_fit, d_y_fit, b_y_fit = x[lim_inf:lim_sup], y[lim_inf:lim_sup], d_y[lim_inf:lim_sup], b_y[lim_inf:lim_sup]
    opt, cov = curve_fit(model, x_fit, y_fit, sigma=d_y_fit, absolute_sigma=True)
    
    n_boot = len(b_y[0])

    x_linsp = np.linspace(x_fit[0], x_fit[-1], 51)
    
    if extension:
        x_linsp=np.linspace(extension[0], extension[1], extension[2])
        
    y_linsp = model(x_linsp, *opt)
        
    b_opt = []
    ## b_c2r = []
    b_y_linsp = []
    
    for j in range(n_boot):
        y_fit_j = [b_y_fit[i][j] for i in range(len(b_y_fit))]
        
        opt_j, cov_j = curve_fit(model, x_fit, y_fit_j, sigma=d_y_fit, absolute_sigma=True)
        b_opt.append(opt_j)

        ## b_c2r.append(reg.chi2_corr(x_fit, y_fit_j, model, np.diag(np.array(d_y_fit)**2), *opt_j))
        
        y_linsp_j = model(x_linsp, *opt_j)
        b_y_linsp.append(y_linsp_j)

    d_opt = np.std(b_opt, axis=0, ddof=1)
    
    c2r = reg.chi2_corr(x_fit, y_fit, model, np.diag(np.array(d_y_fit)**2), *opt)
    ## d_c2r = np.std(b_c2r)
    
    d_y_linsp = np.std(b_y_linsp, axis=0, ddof=1)
    return x_linsp, y_linsp, d_y_linsp, opt, d_opt, b_opt, c2r   

In [17]:
def thermalization(path, *, bc = 'PBC', dis_max = 0):
    '''
    Thermalization of MCMC time-series.
    '''
    os.makedirs(f'{path}/analysis/therm/', exist_ok=True)
    
    output = np.loadtxt(f'{path}/data/dati_0.dat', skiprows=1)
    
    for idx, name in enumerate(['U_s', 'U_t']):
        
        if (bc  == 'OBC'):
            for dis in np.arange(0, dis_max + 1):
                plaq = output[:, 2*dis + idx]
                
                x = np.arange(0,200)
                y = plaq[:200]
                
                plt.figure(figsize=(18,12))
                plt.plot(x, y
                            , marker = 'o'
                            , linestyle = '-', linewidth = 0.375
                            , markersize = 2
                            , color = plot.color_dict[1])
                    
                plt.xlabel(r'$t_i$')

                plt.grid (True, linestyle = '--', linewidth = 0.25)

                plt.savefig(f'{path}/analysis/therm/therm_{name}_dis{dis}.png', bbox_inches='tight')
                plt.close()

        elif (bc == 'PBC'):
            plaq = output[:, idx]
        
            x = np.arange(0,200)
            y = plaq[:200]

            plt.figure(figsize=(18,12))
            plt.plot(x, y
                        , marker = 'o'
                        , linestyle = '-', linewidth = 0.375
                        , markersize = 2
                        , color = plot.color_dict[1])
                
            plt.xlabel(r'$t_i$')

            plt.grid (True, linestyle = '--', linewidth = 0.25)

            plt.savefig(f'{path}/analysis/therm/therm_{name}.png', bbox_inches='tight')
            plt.close()

In [18]:
Nt, Ns, b, bc = 32, 32, 2, 'OBC'

dis_max = 12

input_dir =  f'/home/negro/projects/matching/step_scaling/FVE/Nt{Nt}_Ns{Ns}_b{b}_{bc}_xy'

thermalization(input_dir, bc=bc, dis_max=dis_max)

In [19]:
therm_drop = 200

if not os.path.isfile(f"{input_dir}/analysis/dati.dat"):
    concatenate.concatenate(f"{input_dir}/data", therm_drop, f"{input_dir}/analysis/")

In [20]:
data = readfile(input_dir)

os.makedirs(f'{input_dir}/analysis/bsa/', exist_ok=True)

for idx, name in enumerate(['U_s', 'U_t']):
    plaq = data[:, idx]
    
    block_range, err, d_err = boot.blocksize_analysis_primary(plaq, 500, [2, 500, 2])
    
    plt.figure(figsize=(18,12))
    plt.errorbar(block_range, err, yerr=d_err
                , marker = 'o'
                , linestyle = '-', linewidth = 0.375
                , markersize = 2
                , color = plot.color_dict[1])
    
    plt.xlabel(r'$K$')
    plt.ylabel(r'$\overline{\sigma}_{\overline{F^{(K)}}}$')
    plt.title('Standard error as a function of the blocksize')

    plt.grid(True, which='both', linestyle='--', linewidth=0.25)

    plt.savefig(f'{input_dir}/analysis/bsa/{name}_bsa.png', bbox_inches='tight')
    plt.close()

KeyboardInterrupt: 

In [ ]:
def get_plaqs(path, bc = 'PBC', dis_max = 0, *, seed=8220, samples=500, blocksize=200):
    '''
    Compute plaqs from blocked time-series using bootstrap for error estimation.
    '''
    data = readfile(path)
    
    def id(x):
        return x
    
    for idx, name in enumerate(['U_s', 'U_t']):
        if bc=='OBC':
            for dis in np.arange(0, dis_max+1):
                plaq = data[:, idx + 2*dis]
                        
                _, err, risboot = boot.bootstrap_for_primary(id, plaq, 1, samples, seed=8220, returnboot=True)
                
                np.save(f'{path}/analysis/boot_{name}_dis{dis}', np.array([np.mean(plaq), err, risboot], dtype=object))
        elif bc == 'PBC':
            plaq = data[:, idx]
                        
            _, err, risboot = boot.bootstrap_for_primary(id, plaq, 1, samples, seed=8220, returnboot=True)
            
            np.save(f'{path}/analysis/boot_{name}', np.array([np.mean(plaq), err, risboot], dtype=object))


In [ ]:
get_plaqs(input_dir, bc = bc, dis_max = dis_max, seed=8220, samples=500, blocksize=100)

In [21]:
dis_max = 12

input_dir_PBC =  f'/home/negro/projects/matching/step_scaling/FVE/Nt{Nt}_Ns{Ns}_b{b}_PBC_xy'
input_dir_OBC =  f'/home/negro/projects/matching/step_scaling/FVE/Nt{Nt}_Ns{Ns}_b{b}_OBC_xy'

x = np.arange(Nt, (Nt-2*dis_max) - 1, -2)

types = ['U_s', 'U_t']

color_lines = 4
color_points = 1

fig, ax = plt.subplots(figsize=(16,12))

for idx, name in enumerate(types):
    data_PBC = np.load(f'{input_dir_PBC}/analysis/boot_{name}.npy', allow_pickle=True)
    
    res, d_res, res_bs = map(np.array, data_PBC)
    
    ax.hlines(y=res, xmin=x.min(), xmax=x.max(), **plot.fit(color_lines + idx), label=fr'PBC ${name}$')
    ax.fill_between(x, res-d_res, res+d_res, **plot.conf_band(color_lines + idx))
    
    for dis in np.arange(0, dis_max + 1):
        data_OBC = np.load(f'{input_dir_OBC}/analysis/boot_{name}_dis{dis}.npy', allow_pickle=True)
        res, d_res, res_bs = map(np.array, data_OBC)
        
        ax.errorbar(Ns - 2*dis, res, d_res, **plot.data(color_points + idx), label=fr'OBC ${name}$' if dis==0 else None)
    
## plot details

axins = fig.add_axes([0.475, 0.35, 0.4, 0.3])  # [left, bottom, width, height]

for idx, name in enumerate(types):
    data_PBC = np.load(f'{input_dir_PBC}/analysis/boot_{name}.npy', allow_pickle=True)
    res, d_res, res_bs = map(np.array, data_PBC)
    axins.hlines(y=res, xmin=x.min(), xmax=x.max(), **plot.fit(color_lines + idx))
    axins.fill_between(x, res-d_res, res+d_res, **plot.conf_band(color_lines + idx))
    
    for dis in np.arange(0, dis_max + 1):
        data_OBC = np.load(f'{input_dir_OBC}/analysis/boot_{name}_dis{dis}.npy', allow_pickle=True)
        res, d_res, res_bs = map(np.array, data_OBC)
        axins.errorbar(Ns - 2*dis, res, d_res, **plot.data(color_points + idx))

# zoomed region limits
axins.set_xlim(24.5, 7.5)
axins.set_ylim(0.80596, 0.80601)

axins.yaxis.set_major_formatter(ScalarFormatter())
axins.yaxis.get_major_formatter().set_scientific(False)
axins.yaxis.get_major_formatter().set_useOffset(False)

# add ticks
axins.set_xticks(np.arange(24, 7, -2)) 
axins.set_yticks([0.80596, 0.80599, 0.80601])

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
    

# add grid
axins.grid(True, which='both', linestyle='--', linewidth=0.25)

ax.set_xlabel(r'$N_s$')
ax.set_ylabel(fr'$\langle U_{{s/t}} \rangle$')
ax.grid(True, which='both', linestyle='--', linewidth=0.25)
ax.set_xticks(x)
ax.invert_xaxis()
ax.legend()

plot_name = f'{input_dir}/analysis/plaq_combined_bigaxsnames'

for type in types:
    plot_name += f'_{type}'
    
plt.savefig(f'{plot_name}.png', dpi=600, bbox_inches='tight')
plt.close()